# Overview of Sampling Methods to solve a Linear Regression Problem:

## - Metropolis Hastings Sampling

## - Hamiltonian Monte Carlo

## - Automatic Relevance Detection with HMC

Whilst regression problems can be solved in closed form, we will demonstrate sampling methods to sample from the joint posterior of our hyper-parameters

In [1]:
import time
import numpy              as np
import pandas             as pd
import scipy.stats        as stats
import scipy.spatial      as spatial
import seaborn            as seabornInstance 
import matplotlib.pyplot  as plt
import statsmodels.api    as sm

from sklearn.linear_model import LinearRegression
from sklearn              import metrics
from tqdm.notebook        import tqdm

In [2]:
import sys
sys.path.append('../')

In [3]:
from ML_implementations import *

We will solve a regression problem where we assume a mean zero Gaussian prior on the weights with precision alpha and a mean zero Gaussian prior on the noise with precision beta.

We will work on the Energy Efficiency dataset which has 8 predictors + intercept/bias.

Preprocessing has been performed

In [4]:
with open("ee-train.csv", "rb") as csvfile:
    train = pd.read_csv("ee-train.csv",low_memory=False) 

In [5]:
with open("ee-test.csv", "rb") as csvfile:
    test = pd.read_csv("ee-test.csv",low_memory=False) 

In [6]:
train_means = np.mean(train, axis = 0)
train_std   = np.std(train,axis = 0)

In [7]:
train_norm = (train - train_means)/train_std
train_norm.loc[:,'Heating Load'] = train.loc[:,'Heating Load']
train_norm.insert(0,'Constant',1)
n_features = train_norm.shape[1]
test_norm = (test - train_means)/train_std
test_norm.loc[:,'Heating Load'] = test.loc[:,'Heating Load']
test_norm.insert(0,'Constant',1)

In [8]:
x_train = train_norm.iloc[:,:n_features-1]
y_train = train_norm.iloc[:,n_features-1]
x_test  = test_norm.iloc[:,:n_features-1]
y_test  = test_norm.iloc[:,n_features-1]

### Metropolis - Hastings Sampling

In [9]:
q_star = lambda x: stats.multivariate_normal.rvs(size=1, mean=x, cov = 0.01*np.eye(len(x)))

In [10]:
def p_star(x, X, Y):
    
    alpha     = x[0]
    
    beta      = x[1]
    
    w         = x[2:]
    
    alpha     = np.exp(alpha)
    
    beta      = np.exp(beta)
    
    n,m       = X.shape
    
    first     = -(n/2)*np.log(2*np.pi/beta)  - (beta/2)*((Y - X @ w)**2).sum()
    
    second    = -(m/2)*np.log(2*np.pi/alpha)  - (alpha/2)*(w**2).sum()
    
    pdf       = first + second
    
    return pdf

In [12]:
mh        = sampling.MH(p_star, q_star, 11)

In [13]:
samples   = mh.sample(x_train, y_train)

In [14]:
estimates = mh.estimates

In [15]:
alpha   = np.exp(estimates[0])
beta    = np.exp(estimates[1])
weights = estimates[2:]

We achieve a RMSE on the test of:

In [16]:
np.sqrt(np.mean(np.square(x_test @ weights - y_test)))

2.889639117679848

### Hamiltonian MC

In [17]:
def e_func(x, x_train, y_train):
    
    alpha     = np.exp(x[0])
    
    beta      = np.exp(x[1])
    
    w         = np.array(x[2:])

    n,m       = x_train.shape
    
    mat       = y_train.T @ y_train - w.T @ x_train.T @ y_train - y_train.T @ x_train @ w + w.T @ x_train.T @ x_train @ w
    
    pdf       = -(n/2)*np.log((2*np.pi)/beta)  - (beta/2)*(mat) \
                +(m/2)*np.log(alpha/(2*np.pi)) - (alpha/2)*(w.T @ w)
    
    return -pdf

In [18]:
def e_grad(x, x_train, y_train):
    
    alpha = np.exp(x[0])
    
    beta  = np.exp(x[1])
    
    w     = np.array(x[2:])
    
    n,m   = x_train.shape
    
    g = np.empty(11)
    
    g[0] = 0.5*m - 0.5*alpha*(w.T @ w)
    
    g[1] = 0.5*n - 0.5*beta*((y_train - x_train @ w)**2).sum()
    
    g[2:] = -alpha*w - 0.5*beta*(-(x_train.T @ y_train) \
                                 - (y_train.T @ x_train) \
                                 + 2*(x_train.T @ x_train @ w))
    
    return -g

In [19]:
hmc = sampling.Hamiltonian(e_func, e_grad, 11, epsilon0 = 0.001)

In [20]:
samples = hmc.sample(x_train, y_train)

../ML_implementations/sampling/sampling.py:173: RuntimeWarning: overflow encountered in exp
  if np.random.uniform() < np.exp(threshold):


KeyboardInterrupt: 

In [ ]:
alpha   = np.exp(samples[-1,0])
beta    = np.exp(samples[-1,1])
weights = samples[-1,2:]

We achieve a RMSE on the test of:

In [ ]:
np.sqrt(np.mean(np.square(x_test @ weights - y_test)))

### Automatic Relevance Detection

We can change our prior on the weights to a Gaussian prior for each weight with different precision hyper-parametrs. We can then sample from this larger joint posterior to determine which features are relevant. High precision indicated low variance and therefore weights likely close to zero (mean zero prior assumed).

In [ ]:
def e_func(x, x_train, y_train):
    
    alphas     = np.exp(x[0:9])
    
    beta      = np.exp(x[9])
    
    w         = np.array(x[10:])

    n,m       = x_train.shape
    
    mat       = y_train.T @ y_train - w.T @ x_train.T @ y_train \
                - y_train.T @ x_train @ w + w.T @ x_train.T @ x_train @ w
    
    pdf       = -(n/2)*np.log((2*np.pi)/beta)   - (beta/2)*(mat) \
                + 0.5*np.sum(np.log(alphas)) - 0.5*np.sum(alphas*(w**2)) - (m/2)*np.log(2*np.pi)
    
    return -pdf

In [ ]:
def e_grad(x, x_train, y_train):
    
    alphas = np.exp(x[0:9])
    
    beta  = np.exp(x[9])
    
    w     = np.array(x[10:])
    
    n,m   = x_train.shape
    
    g     = np.empty(19)
    
    g[0:9]  = (1/2)*(1 - alphas*(w**2))
    
    g[9]    = 0.5*n - 0.5*beta*((y_train - x_train @ w)**2).sum()
    
    g[10:]  = -alphas*w - 0.5*beta*(-(x_train.T @ y_train) \
                                 - (y_train.T @ x_train) \
                                 + 2*(x_train.T @ x_train @ w))
    
    return -g

In [ ]:
hmc_ard = sampling.Hamiltonian(e_func, e_grad, 19, epsilon0 = 0.001)

In [ ]:
ard_samples = hmc_ard.sample(x_train,y_train)

In [ ]:
alpha   = np.exp(ard_samples[-1,0:9])
beta    = np.exp(ard_samples[-1,9])
weights = ard_samples[-1,10:]

We achieve a RMSE on the test of:

In [ ]:
np.sqrt(np.mean(np.square(x_test @ weights - y_test)))

With alphas of:

In [ ]:
alpha

And therefore features variances of:

In [ ]:
1/alpha